In [287]:
model_inputs = {
    "profitability": {
        "class_weight": 0.30,
        "weights": [1.0], 
        "metrics": {
            "oper_margin": {
                "lower_is_better": False,
                "thresholds": [
                    (40, float("inf")),
                    (35, 39),
                    (30, 34),
                    (25, 29),
                    (20, 24),
                    (15, 19),
                    (10, 14),
                    (5, 9),
                    (float("-inf"), 0)
                ],
            }
        },
    },
    "leverage_coverage": {
        "class_weight": 0.55,
        "weights": [0.4, 0.3, 0.3],
        "metrics": {
            "tot_debt_to_tot_eqy": {
                "lower_is_better": True,
                "thresholds": [
                     (float("-inf"), 2.0),
                     (2.0, 16.0),
                     (16.0, 24.0),
                     (24.0, 33.0),
                     (33.0, 43.0),
                     (43.0, 54.0),
                     (54.0, 68.0),
                     (68.0, 94.0),
                     (94.0, float("inf")),
                ],
            },
            "tot_debt_to_ebitda": {
                "lower_is_better": True,
                "thresholds": [
                    (float("-inf", 0.09),
                    (0.09, 0.49),
                    (0.49, 0.9),
                    (0.9, 1.36),
                    (1.36, 1.68),
                    (1.68, 2.26),
                    (2.26, 3.27),
                    (3.27, 4.4),
                    (4.4, float("inf")),
                ],
            },
            "ebitda_to_tot_int_exp": {
                "lower_is_better": False,
                "thresholds": [
                    (25, float("inf")),
                    (20, 25),
                    (15, 20),
                    (10, 15),
                    (5, 10),
                    (3, 5),
                    (1, 3),
                    (0, 1),
                    (float("-inf"), 0),
                ],
            },
        },
    },
    "efficiency": {
        "class_weight": 0.15,
        "weights": [0.5, 0.5],
        "metrics": {
            "return_on_asset": {
                "lower_is_better": False,
                "thresholds": [
                    (0.15, float("inf")),
                    (0.10, 0.15),
                    (0.08, 0.10),
                    (0.06, 0.08),
                    (0.04, 0.06),
                    (0.02, 0.04),
                    (0.00, 0.02),
                    (-0.02, 0.00),
                    (float("-inf"), -0.02)
                ],
            },
            "asset_turnover": {
                "lower_is_better": False,
                "thresholds": [
                    (4.0, float("inf")),
                    (3.0, 4.0),
                    (2.0, 3.0),
                    (1.5, 2.0),
                    (1.0, 1.5),
                    (0.75, 1.0),
                    (0.5, 0.75),
                    (0.25, 0.5),
                    (float("-inf"), 0.25)
                ],
            },
        },
    },
}

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 43 (1299856514.py, line 52)

In [288]:
class CreditRatingCalculator:
    def __init__(self, metrics):
        self.metrics = metrics
        
    def _calculate_metric_score(self, metric, thresholds, inverse):
        for score, (lower, upper) in enumerate(thresholds, start=1):
            if (inverse and metric <= upper) or (not inverse and metric >= lower):
                return score
        return len(thresholds) // 2 # else return the middle score

    def _calculate_category_score(self, category_metrics, ratios):
        total_weighted_score = 0

        for metric, weight in zip(
            category_metrics["metrics"].items(), category_metrics["weights"]
        ):
            metric_name, metric_data = metric
            value = ratios[metric_name]
            score = self._calculate_metric_score(
                value, metric_data["thresholds"], metric_data["lower_is_better"]
            )
            total_weighted_score += score * weight

        return total_weighted_score

    def _calculate_scores(self, ratios):
        scores = {}
        for category, category_data in self.metrics.items():
            category_score = self._calculate_category_score(category_data, ratios)
            scores[category] = category_score
        return scores

    def _calculate_weighted_score(self, scores):
        weights = {
            category: category_data["class_weight"]
            for category, category_data in self.metrics.items()
        }
        print(weights)
        print(scores)
        return sum(scores[category] * weight for category, weight in weights.items())

    def _determine_credit_rating(self, weighted_score):
        # credit_ratings = [
        #     (1.5, "Aaa"),
        #     (2.5, "Aa"),
        #     (3.5, "A"),
        #     (4.5, "Baa"),
        #     (5.5, "Ba"),
        #     (6.5, "B"),
        #     (7.5, "Caa"),
        #     (8.5, "Ca"),
        #     (float("inf"), "C"),
        # ]
        
        credit_ratings = [
            (2.5, "Aaa"),
            (3.5, "Aa"),
            (4.5, "A"),
            (5.5, "Baa"),
            (6.5, "Ba"),
            (7.5, "B"),
            (8.5, "Caa"),
            (9.5, "Ca"),
            (float("inf"), "C"),
        ]

        for threshold, rating in credit_ratings:
            if weighted_score < threshold:
                return rating

    def calculate_credit_rating(self, ratios):
        self.scores = self._calculate_scores(ratios)
        self.credit_score = self._calculate_weighted_score(self.scores)
        self.credit_rating = self._determine_credit_rating(self.credit_score)

In [283]:
company = "MTN SJ Equity"

In [284]:
model_metrics = [
    "oper_margin", 
    "tot_debt_to_tot_eqy", 
    "tot_debt_to_ebitda", 
    "ebitda_to_tot_int_exp", 
    "return_on_asset",
    "asset_turnover",
]

ratios = metrics[company].loc[model_metrics].to_dict()

In [285]:
ratios

{'oper_margin': 23.90386585763219,
 'tot_debt_to_tot_eqy': 65.11678489099474,
 'tot_debt_to_ebitda': 1.305328924586783,
 'ebitda_to_tot_int_exp': 13.11412971213706,
 'return_on_asset': 7.405180900886093,
 'asset_turnover': 0.6140331573693389}

In [286]:
model = CreditRatingCalculator(model_inputs)
model.calculate_credit_rating(ratios)
print(f"Class Scoring: {model.scores}")
print(f"Credit Score: {model.credit_score}")
print(f"Credit Rating: {model.credit_rating}")

{'profitability': 0.3, 'leverage_coverage': 0.55, 'efficiency': 0.15}
{'profitability': 5.0, 'leverage_coverage': 5.5, 'efficiency': 4.0}
Class Scoring: {'profitability': 5.0, 'leverage_coverage': 5.5, 'efficiency': 4.0}
Credit Score: 5.125
Credit Rating: Baa


In [282]:
eq = metrics.loc["ebitda_to_tot_int_exp"].remove_outliers()
qq = eq.quantile(np.linspace(0.01, 1, 9))

In [260]:
qq = qq.round(2)

In [261]:
for val in qq:
    
    a = eq[eq < val]
    print(len(a))

1
14
26
38
50
63
75
87
99


In [267]:
import numpy as np

values = qq.values #np.array([2., 16., 24., 33., 43., 54., 68., 94., 207.])
sorted_values = np.sort(np.unique(values))

thresholds = []
prev_value = float("-inf")

for value in sorted_values:
    thresholds.append((prev_value, value))
    prev_value = value

thresholds.append((sorted_values[-1], float("inf")))

In [268]:
thresholds

[(-inf, 0.36),
 (0.36, 3.84),
 (3.84, 6.29),
 (6.29, 8.26),
 (8.26, 9.97),
 (9.97, 15.48),
 (15.48, 20.9),
 (20.9, 69.63),
 (69.63, 278.83),
 (278.83, inf)]

In [270]:
import numpy as np

values = qq.values #np.array([2., 16., 24., 33., 43., 54., 68., 94., 207.])
sorted_values = np.sort(np.unique(values))[::-1]  # Reverse the order

thresholds = []
prev_value = float("inf")

for value in sorted_values:
    thresholds.append((value, prev_value))
    prev_value = value

thresholds.append((float("-inf"), sorted_values[-1]))

In [271]:
thresholds

[(278.83, inf),
 (69.63, 278.83),
 (20.9, 69.63),
 (15.48, 20.9),
 (9.97, 15.48),
 (8.26, 9.97),
 (6.29, 8.26),
 (3.84, 6.29),
 (0.36, 3.84),
 (-inf, 0.36)]

In [197]:
bounds

[(1.6923229820685262, inf),
 (0, 206.8655490463593),
 (206.8655490463593, 93.77429513141182),
 (93.77429513141182, 67.82813253514031),
 (67.82813253514031, 53.69410519913799),
 (53.69410519913799, 43.34814127903013),
 (43.34814127903013, 32.61397408917929),
 (32.61397408917929, 23.820047331122783),
 (23.820047331122783, 15.84466213095929),
 (15.84466213095929, 1.6923229820685262)]

In [ ]:
np.nextafter(

In [17]:
import pandas as pd 

In [18]:
from quantstats import extend_pandas
extend_pandas()

In [35]:
df = pd.read_csv("research/JALSH Index_dataset_2000_2024_clean.csv", index_col=0, header=[0, 1])
classfier = pd.read_excel("research/classification_data.xlsx", index_col=0)
metrics = pd.read_excel("research/metrics_full.xlsx", index_col=0)

In [20]:
def filter_securities(filters, data=classfier):
    
    # Apply filters directly on the transposed
    filtered_data = data.query(
        ' and '.join(f'`{k}` == "{v}"' for k, v in filters.items())
    )
    
    return list(filtered_data.index)

In [21]:
classfier.sector.unique()

array(['Financial', 'Consumer, Non-cyclical', 'Industrial',
       'Basic Materials', 'Diversified', 'Consumer, Cyclical',
       'Communications', 'Technology', 'Energy'], dtype=object)